# Create Files

You simply create files by creating a channel

A channel consists of:

**file name + header + calibration + atss data**

NONE of the tags is *repeated* 

"run" appears in the filename BUT NOT inside the JSON header *again*

The **read_atssheader(filename)** function (without extension):

* reads the filename
* reads the header
* checks the file size / samples from the .atss file

and constructs a channel object.

If now data is created and put inside the atss file - nothing is to do afterwards.

When later opening files, the samples will be calculated automatically.


In [1]:
import json
import sys, os
import numpy as np
import struct
sys.path.append(os.path.realpath('../../include'))
import json_header as jh

channel_2 = jh.channel()            # create a channel
channel_2['serial'] = 803
channel_2['sample_rate'] = 256.0
channel_2['channel_no'] = 2
channel_2['channel_type'] = "Hx"
channel_2['run'] = 907
channel_2['system'] = "ADU-08e"
channel_2['date'] = "2021-05-19"
channel_2['time'] = "17:32:04"
channel_2['latitude'] = 52.2443
channel_2['longitude'] = 10.55
channel_2['elevation'] = 78.9
channel_2['units'] = "mV"

# file name extract from the channel
filename = jh.atss_filename(channel_2)
print(filename)
print("serial_system_Cchannel_Rrun,sample_rateHz");


803_ADU-08e_C002_R907_THx_256Hz
serial_system_Cchannel_Rrun,sample_rateHz


In [2]:
# what is not inside the filename - header from channel
header = jh.json_header(channel_2)
print(header)

{'date': '2021-05-19', 'time': '17:32:04', 'fracs': 0.0, 'latitude': 52.2443, 'longitude': 10.55, 'elevation': 78.9, 'angle': 0.0, 'dip': 0.0, 'units': 'mV', 'source': ''}


In [3]:
# and as you would write as JSON to file
print(json.dumps(header, indent=2, sort_keys=False, ensure_ascii=False))

{
  "date": "2021-05-19",
  "time": "17:32:04",
  "fracs": 0.0,
  "latitude": 52.2443,
  "longitude": 10.55,
  "elevation": 78.9,
  "angle": 0.0,
  "dip": 0.0,
  "units": "mV",
  "source": ""
}


In [4]:
# let's write the header to the disk
# the header contains an EMPTY calibration
jh.write_atssheader(channel_2)

fu
{'serial': 803, 'system': 'ADU-08e', 'channel_no': 2, 'run': 907, 'channel_type': 'Hx', 'sample_rate': 256.0, 'date': '2021-05-19', 'time': '17:32:04', 'fracs': 0.0, 'latitude': 52.2443, 'longitude': 10.55, 'elevation': 78.9, 'angle': 0.0, 'dip': 0.0, 'units': 'mV', 'source': '', 'sensor_calibration': {'sensor': '', 'serial': 0, 'chopper': 0, 'units_amplitude': 'mV/nT', 'units_frequency': 'Hz', 'units_phase': 'degrees', 'date': '1970-01-01', 'time': '00:00:00', 'Operator': '', 'f': [0.0], 'a': [0.0], 'p': [0.0]}}


In [5]:
# so you want to create an electric field for Ex?
channel_2['serial'] = 804
channel_2['sample_rate'] = 256.0
channel_2['channel_no'] = 0
channel_2['channel_type'] = "Ex"
channel_2['run'] = 907
channel_2['system'] = "ADU-08e"
channel_2['date'] = "2021-05-19"
channel_2['time'] = "17:32:04"
channel_2['angle'] = 0.0                # North
channel_2['dip'] = 0.0                  # horizontal, default
channel_2['units'] = "mV/km"            # DO NOT FORGET THIS
# the following lines prevent questions
# even though calibration is empty
channel_2['sensor_calibration']['sensor'] = "Electrode"
channel_2['sensor_calibration']['units_amplitude'] = "mV"
print(json.dumps(channel_2, indent=2, sort_keys=False, ensure_ascii=False))
jh.write_atssheader(channel_2)
print(json.dumps(channel_2, indent=2, sort_keys=False, ensure_ascii=False))

newfile = jh.atss_filename(channel_2)
print("written: ", newfile)


{
  "serial": 804,
  "system": "ADU-08e",
  "channel_no": 0,
  "run": 907,
  "channel_type": "Ex",
  "sample_rate": 256.0,
  "date": "2021-05-19",
  "time": "17:32:04",
  "fracs": 0.0,
  "latitude": 52.2443,
  "longitude": 10.55,
  "elevation": 78.9,
  "angle": 0.0,
  "dip": 0.0,
  "units": "mV/km",
  "source": "",
  "sensor_calibration": {
    "sensor": "Electrode",
    "serial": 0,
    "chopper": 0,
    "units_amplitude": "mV",
    "units_frequency": "Hz",
    "units_phase": "degrees",
    "date": "1970-01-01",
    "time": "00:00:00",
    "Operator": "",
    "f": [
      0.0
    ],
    "a": [
      0.0
    ],
    "p": [
      0.0
    ]
  }
}
fu
{'serial': 804, 'system': 'ADU-08e', 'channel_no': 0, 'run': 907, 'channel_type': 'Ex', 'sample_rate': 256.0, 'date': '2021-05-19', 'time': '17:32:04', 'fracs': 0.0, 'latitude': 52.2443, 'longitude': 10.55, 'elevation': 78.9, 'angle': 0.0, 'dip': 0.0, 'units': 'mV/km', 'source': '', 'sensor_calibration': {'sensor': 'Electrode', 'serial': 0, 'c

In [6]:
# create some "fake" data with 1024 samples
ts = np.linspace(0, 3.1415, 1024)
data   = np.sin(ts)
try:
    fo = open(filename + ".atss", 'wb')
except IOError:
    raise Exception(f'unable to open file for writing: {filename}')
try:
    with open(filename + ".atss", 'rb') as f:
        for d in data:
            fo.write(struct.pack('d', d))
    fo.close()
except Exception:
    raise Exception(f'unable to write atss file: {filename}')
    

In [7]:
# re-read the channel created above into a new one
channel_new = jh.read_atssheader(filename)
print(channel_new)

{'serial': 803, 'system': 'ADU-08e', 'channel_no': 2, 'run': 907, 'channel_type': 'Hx', 'sample_rate': 256.0, 'date': '2021-05-19', 'time': '17:32:04', 'fracs': 0.0, 'latitude': 52.2443, 'longitude': 10.55, 'elevation': 78.9, 'angle': 0.0, 'dip': 0.0, 'units': 'mV', 'source': '', 'sensor_calibration': {'sensor': '', 'serial': 0, 'chopper': 0, 'units_amplitude': 'mV/nT', 'units_frequency': 'Hz', 'units_phase': 'degrees', 'date': '1970-01-01', 'time': '00:00:00', 'Operator': '', 'f': [0.0], 'a': [0.0], 'p': [0.0]}, 'samples': 1024}


In [8]:
# samplesare NOT part of the header
# but read on the fly - remind we have created 1024 samples above!
print("samples from fake data above: ", channel_new["samples"])

samples from fake data above:  1024


In [9]:
# you can prove that the header itself does not contain samples
header_new = jh.json_header(channel_new)
print(json.dumps(header_new, indent=2, sort_keys=False, ensure_ascii=False))

{
  "date": "2021-05-19",
  "time": "17:32:04",
  "fracs": 0.0,
  "latitude": 52.2443,
  "longitude": 10.55,
  "elevation": 78.9,
  "angle": 0.0,
  "dip": 0.0,
  "units": "mV",
  "source": ""
}


# Summary

It is difficult to understand why the header as JSON does **NOT** repeat items which are already encoded in the file name or contains the *varying* file size.

In programming redundant variables are causing unresolvable problems:

start_time, stop_time, sample_rate and samples - is a good example.

The stop_time can always be calculated from start_time, sample_rate and samples.

If one of the processes "forgets" to update the stop_time the data is in an usuable state. If accidently a calculation used N-1 instead of N (typical vector problem where 1023 is the last index of 1024 items) this happens.

In the examples above, samples is secure - that is what is really there, and is calculated from the file size. In case the file size is truncated by an FIR filter, you can (worst case) try to adjust the start_time only.

I expect that some users will try Python or other libraries - and they will not be able to handle all data correctly. Therefor the above design is not only correct - it is also robust.


In [10]:
# create a CEA obs for 营口台 station
channel_cea = jh.channel()            # create a channel
channel_cea['serial'] = 22
channel_cea['sample_rate'] = 4096.0
channel_cea['channel_no'] = 2
channel_cea['channel_type'] = "Hy"
channel_cea['run'] = 8               # slice is treated as run
channel_cea['system'] = "ADU-07e"
channel_cea['date'] = "2019-02-07"
channel_cea['time'] = "22:00:00"
channel_cea['latitude'] = 39.9042
channel_cea['longitude'] = 16.4074
channel_cea['elevation'] = 44.2
channel_cea['units'] = "mV"
channel_cea['site'] = "21007"        # remind that site is defined as string!
channel_cea['source'] = "ns"         # natural source run

cea_filename = jh.cea_atss_filename(channel_cea)
print("file name CEA style: ", cea_filename)
print("slice 8 appears as run!")



file name CEA style:  cea_site_20190207_ns_4096Hz_Hy_R008
slice 8 appears as run!


In [11]:
# create flight data from Motus IMU unit
channel_imu = jh.channel()            # create a channel
channel_imu['serial'] = 2522
channel_imu['sample_rate'] = 8192.0
channel_imu['channel_no'] = 0
channel_imu['channel_type'] = "Pitch"
channel_imu['run'] = 0              
channel_imu['system'] = "Motus-P2"
channel_imu['date'] = "2020-09-14"
channel_imu['time'] = "11:00:00"
channel_imu['latitude'] = 53.4932
channel_imu['longitude'] = 8.7154
channel_imu['elevation'] = 14.2
channel_imu['units'] = "deg"
flight_name = jh.atss_filename(channel_imu)
print ("flight data example: ", flight_name)

flight data example:  2522_Motus-P2_C000_R000_TPitch_8192Hz


In [12]:
# create temperature data from fluxgate sensor (FGS-03e, FGS-04e)
# can be read out by I2C bus
channel_fgst = jh.channel()            # create a channel
channel_fgst['serial'] = 137
channel_fgst['sample_rate'] = 8192.0
channel_fgst['channel_no'] = 11
channel_fgst['channel_type'] = "T"     # take C(elsius) or K(elvin) 
channel_fgst['run'] = 0              
channel_fgst['system'] = "ADU-07e"
channel_fgst['date'] = "2020-09-14"
channel_fgst['time'] = "11:00:00"
channel_fgst['latitude'] = 17.3850
channel_fgst['longitude'] = 78.4867
channel_fgst['elevation'] = 506
channel_fgst['units'] = "deg"
channel_fgst['site'] = "హైదరాబాద్"       # check if Hyderabad gets written correctly

temperature_name = jh.atss_filename(channel_fgst)
print ("temperature data example: ", temperature_name)

temperature data example:  137_ADU-07e_C011_R000_TT_8192Hz
